# 7 Customizing Built-Ins: UserString, UserList, and UserDict

Sometimes you need to customize built-in types, such as strings, lists, and dictionaries to add and modify certain behavior. Since Python 2.2, you can do that by subclassing those types directly. However, you could face some issues with this approach, as you’ll see in a minute.

Python’s collections provides three convenient wrapper classes that mimic the behavior of the built-in data types:

- UserString
- UserList
- UserDict

With a combination of regular and special methods, you can use these classes to mimic and customize the behavior of strings, lists, and dictionaries.

Nowadays, developers often ask themselves if there’s a reason to use UserString, UserList, and UserDict when they need to customize the behavior of built-in types. The answer is yes.

Built-in types were designed and implemented (https://www.youtube.com/watch?v=heJuQWNdwJI) with the open-closed principle(https://en.wikipedia.org/wiki/Open%E2%80%93closed_principle) in mind. This means that they’re open for extension but closed for modification. Allowing modifications on the core features of these classes can potentially break their invariants. So, Python core developers decided to protect them from modifications.

For example, say you need a dictionary that automatically lowercases the keys when you insert them. You could subclass dict and override .__setitem__() so every time you insert a key, the dictionary lowercases the key name

## 7.1 Extends normal Dict class

In [1]:
class LowerDict(dict):
    def __setitem__(self, key, value):
        key = key.lower()
        super().__setitem__(key, value)

In [3]:
ordinals = LowerDict({"FIRST": 1, "SECOND": 2})
print(ordinals)

{'FIRST': 1, 'SECOND': 2}


In [5]:
ordinals["THIRD"] = 3
print(ordinals)

{'FIRST': 1, 'SECOND': 2, 'third': 3}


In [11]:
# As the custom class extends from the Dict, so it's an instance of dict
isinstance(ordinals, dict)

True

You can notice that this dictionary works correctly when you insert new keys using dictionary-style assignment with square brackets ([]). However, it doesn’t work when you pass an initial dictionary to the class constructor or when you use .update(). This means that you would need to **override .__init__(), .update(), etc.**, for your custom dictionary to work correctly.

## 7.2 Extends UserDict

Now let's try with UserDict class

In [6]:
from collections import UserDict


class LowerUserDict(UserDict):
    def __setitem__(self, key, value):
        key = key.lower()
        super().__setitem__(key, value)

In [7]:
user_dict_lower = LowerUserDict({"FIRST": 1, "SECOND": 2})
print(user_dict_lower)

{'first': 1, 'second': 2}


In [9]:
user_dict_lower["THIRD"] = 3
print(user_dict_lower)

{'first': 1, 'second': 2, 'third': 3}


In [10]:
user_dict_lower.update({"FOUR": 4})
print(user_dict_lower)

{'first': 1, 'second': 2, 'third': 3, 'four': 4}


It works! Your custom dictionary now converts all the new keys into lowercase letters before inserting them into the dictionary.

Note that since you don’t inherit from dict directly, your class doesn’t return instances of dict as in the example above.

In [13]:

print(isinstance(user_dict_lower, UserDict))
print(isinstance(user_dict_lower, dict))

True
False


UserDict stores a regular dictionary in an instance attribute called .data. Then it implements all its methods around that dictionary.

In [14]:
internal_dict = user_dict_lower.data

print(type(internal_dict))
print(internal_dict)

<class 'dict'>
{'first': 1, 'second': 2, 'third': 3, 'four': 4}


## 7.3  UserList and UserString

UserList and UserString work the same way as UserDict. But their .data attribute holds a list and a str object, respectively.

If you need to customize either of these classes, then you just need to override the appropriate methods and change what they do as required.

## 7.4 When to use

In general, you should use UserDict, UserList, and UserString when you need a class that acts almost identically to the underlying wrapped built-in class and you want to customize some part of its standard functionalities.

Another reason to use these classes rather than the built-in equivalent classes is to access the underlying .data attribute to manipulate it directly.

The ability to inherit from built-in types directly has largely superseded the use of UserDict, UserList, and UserString. However, the internal implementation of built-in types makes it hard to safely inherit from them without rewriting a significant amount of code. In most cases, it’s safer to use the appropriate class from collections. It’ll save you from several issues and weird behaviors.